# Ein KI-Chatbot in 5 Minuten

Dieses Notebook führt dich durch folgende Schritte:

- Installation der benötigten Pakete
- Benutzen des Sitemap Loaders um die deepshore.de nach Wissensbeiträgen zu durchkämmen
- Die Dokumente an ein Embeddings Modell schicken, in Vektoren verwandeln und im Index speichern 
- Eine Frage an das Embeddings Modell schicken, die neu gewonnenen Vektoren zusammen mit den indexierten Vektoren verorten und eine Antwort bekommen

Du benötigst:

- python 3.10
- Jupyter Notebook Server
- Einen OpenAI API Token


## Installation

In [4]:
%pip install llama_index=="0.6.38"
%pip install llama_hub=="0.0.26"
%pip install mercury=="2.3.4"

import mercury as mr
 
# set Application parameters
app = mr.App(title="",
        description="",
        show_code=False,
        show_prompt=False,
        continuous_update=True,
        static_notebook=False,
        show_sidebar=True,
        full_screen=True,
        allow_download=True)

  Obtaining dependency information for sqlalchemy>=2.0.15 from https://files.pythonhosted.org/packages/3b/7f/9a11e808fdf1187c8206f204352fbbd0d72b68d6bc8233121058f8bde73d/SQLAlchemy-2.0.20-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Using cached SQLAlchemy-2.0.20-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.4 kB)
Using cached SQLAlchemy-2.0.20-cp310-cp310-macosx_11_0_arm64.whl (2.0 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.27
    Uninstalling SQLAlchemy-1.4.27:
      Successfully uninstalled SQLAlchemy-1.4.27
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mercury 2.3.4 requires sqlalchemy==1.4.27, but you have sqlalchemy 2.0.20 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for llama_hub==0.0.26 from https://files.pythonhosted.org/packages

mercury.App

## Open AI Konfiguration

### Warum OpenAI?

[llama_index](https://github.com/jerryjliu/llama_index) und andere LLM-App-Frameworks benutzen meist als Standard die OpenAI high-level API für Embeddings (Vektorisierung von Daten).

![](images/chatbot_graph-white-bg-v3.png)

In [1]:
import os
import logging
import sys
import getpass

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Dokumente laden

 - Die Dokumente werden geladen, indem man den [Sitemap Loader](https://llama-hub-ui.vercel.app/l/web-sitemap) benutzt, um die Webseite deepshore.de nach Wissensbeiträgen zu durchkämmen

In [2]:
from llama_hub.web.sitemap.base import SitemapReader

import nest_asyncio
nest_asyncio.apply()

loader = SitemapReader(html_to_text=True)
documents = loader.load_data(sitemap_url='https://deepshore.de/sitemap.xml', filter='https://deepshore.de/knowledge')

print(len(documents))

INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.
86


## Die Dokumente an ein Embeddings Modell schicken

 - in Vektoren verwandeln und im Index speichern 

### Warum ein Vektorindex?
ein Vektorindex speichert bzw. indexiert Vektordaten für eine schnellere Abfrage oder Filterung (similarity search) von Daten, die sonst jedes mal gegen das ursprüngliche Modell gesendet werden müssen.
   
![](images/how-does-vector-store-work.png)

In [3]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist()

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 229163 tokens
> [build_index_from_nodes] Total embedding token usage: 229163 tokens


## Eine Frage an das Modell schicken

 - Eine Frage stellen und eine Antwort im Kontext der Daten bekommen

In [ ]:
import textwrap

question = mr.Text(value="Was ist k6.io? Wofür benutzt man es?", label="Womit kann Deepshore helfen?", rows=1)

query_engine = index.as_query_engine()

response = query_engine.query(question.value)

messages = [question.value, "\n".join(textwrap.wrap(response.response,29))]
mr.Chat(messages)